<a href="https://colab.research.google.com/github/ounospanas/KneE-PAD/blob/main/KneE_PAD_Download_Read_and_Segment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [KneE-PAD](https://zenodo.org/records/12112951) repository

In [5]:
# Download KneE-PAD
!wget https://zenodo.org/records/12112951/files/dataset.zip
!unzip dataset.zip

Streaming output truncated to the last 5000 lines.
   creating: dataset/Subject_17/0/Trial_12/
  inflating: dataset/Subject_17/0/Trial_12/emg.npy  
  inflating: dataset/Subject_17/0/Trial_12/imu.npy  
   creating: dataset/Subject_17/0/Trial_13/
  inflating: dataset/Subject_17/0/Trial_13/emg.npy  
  inflating: dataset/Subject_17/0/Trial_13/imu.npy  
   creating: dataset/Subject_17/0/Trial_14/
  inflating: dataset/Subject_17/0/Trial_14/emg.npy  
  inflating: dataset/Subject_17/0/Trial_14/imu.npy  
   creating: dataset/Subject_17/0/Trial_15/
  inflating: dataset/Subject_17/0/Trial_15/emg.npy  
  inflating: dataset/Subject_17/0/Trial_15/imu.npy  
   creating: dataset/Subject_17/0/Trial_16/
  inflating: dataset/Subject_17/0/Trial_16/emg.npy  
  inflating: dataset/Subject_17/0/Trial_16/imu.npy  
   creating: dataset/Subject_17/0/Trial_17/
  inflating: dataset/Subject_17/0/Trial_17/emg.npy  
  inflating: dataset/Subject_17/0/Trial_17/imu.npy  
   creating: dataset/Subject_17/0/Trial_18/
  inf

In [8]:
import os
import numpy as np
from skimage.util import view_as_windows

In [6]:
path = 'dataset/'
window = 4 #seconds
emg_sr = 1259.2592592592594 #Hz
imu_sr = 148.14814814814815 #Hz
window_step = 1/4 #0.25sec
n_sensors = 8

Instead of using np.concatenate or np.append which are very inneficient (takes several minutes) we will count the number of samples/segments to create an empty np.array with zeros and populate later (takes few seconds).

In [9]:
counter = 0

for sub in os.listdir(path):
    print(sub)
    s = int(''.join([s for s in [*sub] if s.isdigit()]))
    for act in os.listdir(path+sub):
        for tr in os.listdir(path+sub+'/'+act):
            emg = np.load(path+sub+'/'+act+'/'+tr+'/'+'emg.npy')
            imu = np.load(path+sub+'/'+act+'/'+tr+'/'+'imu.npy')
            duration = emg.shape[1]/emg_sr
            if int(act)>5:
                # avoid having several walking examples we use a larger stride
                window_step = 1/2
            else:
                window_step = 1/4

            if duration<window:
                emg_segments = np.zeros((1,n_sensors,int(np.round(window*emg_sr))))
                emg_segments[:,:,:emg.shape[1]] = emg

                imu_segments = np.zeros((1,6*n_sensors,int(np.round(window*imu_sr))))
                imu_segments[:,:,:imu.shape[1]] = imu
            else:
                emg_segments = view_as_windows(np.array(emg), window_shape=(n_sensors,int(np.round(window*emg_sr))),
                                        step=int(emg_sr*window_step))[0]

                imu_segments = view_as_windows(np.array(imu), window_shape=(6*n_sensors,int(np.round(window*imu_sr))),
                                        step=int(imu_sr*window_step))[0]

            ### Check if EMG shape matches the IMU shape!
            assert len(emg_segments)==len(imu_segments)
            counter+=len(emg_segments)


Subject_12
Subject_29
Subject_17
Subject_7
Subject_11
Subject_9
Subject_22
Subject_25
Subject_20
Subject_10
Subject_8
Subject_21
Subject_31
Subject_14
Subject_24
Subject_3
Subject_6
Subject_18
Subject_23
Subject_30
Subject_2
Subject_15
Subject_19
Subject_16
Subject_27
Subject_5
Subject_1
Subject_26
Subject_13
Subject_4
Subject_28


In [10]:
# instatiate two tensors with zeros to be filled later with IMU and EMG segments
global_emg_segments = np.zeros((counter,n_sensors,int(window*emg_sr)))
global_imu_segments = np.zeros((counter,6*n_sensors,int(np.round(window*imu_sr))))

subjects = []
labels = []
sessions = []
current_seg_id = 0

for sub in os.listdir(path):
    print(sub)
    s = int(''.join([s for s in [*sub] if s.isdigit()]))
    for act in os.listdir(path+sub):
        for tr in os.listdir(path+sub+'/'+act):
            emg = np.load(path+sub+'/'+act+'/'+tr+'/'+'emg.npy')
            imu = np.load(path+sub+'/'+act+'/'+tr+'/'+'imu.npy')
            duration = emg.shape[1]/emg_sr
            if int(act)>5:
                # avoid having several walking examples we use a larger stride
                window_step = 1/2
            else:
                window_step = 1/4

            if duration<window:
                emg_segments = np.zeros((1,n_sensors,int(np.round(window*emg_sr))))
                emg_segments[:,:,:emg.shape[1]] = emg

                imu_segments = np.zeros((1,6*n_sensors,int(np.round(window*imu_sr))))
                imu_segments[:,:,:imu.shape[1]] = imu
            else:
                emg_segments = view_as_windows(np.array(emg), window_shape=(n_sensors,int(np.round(window*emg_sr))),
                                        step=int(emg_sr*window_step))[0]

                imu_segments = view_as_windows(np.array(imu), window_shape=(6*n_sensors,int(np.round(window*imu_sr))),
                                        step=int(imu_sr*window_step))[0]

            next_seg_id = current_seg_id + len(emg_segments)
            global_emg_segments[current_seg_id:next_seg_id,:,:] = emg_segments
            global_imu_segments[current_seg_id:next_seg_id,:,:] = imu_segments

            subjects+=[s]*len(emg_segments)
            labels+=[int(act)]*len(emg_segments)
            sessions+=[path+sub+'/'+act+'/'+tr]*len(emg_segments)

            current_seg_id += len(emg_segments)

Subject_12
Subject_29
Subject_17
Subject_7
Subject_11
Subject_9
Subject_22
Subject_25
Subject_20
Subject_10
Subject_8
Subject_21
Subject_31
Subject_14
Subject_24
Subject_3
Subject_6
Subject_18
Subject_23
Subject_30
Subject_2
Subject_15
Subject_19
Subject_16
Subject_27
Subject_5
Subject_1
Subject_26
Subject_13
Subject_4
Subject_28


### Save in .npy format (Python)

In [11]:
assert global_emg_segments[1:].shape[0]==global_imu_segments[1:].shape[0]

In [12]:
# save the whole dataset to .npy array
np.save('emg_all.npy',global_emg_segments)
np.save('imu_all.npy',global_imu_segments)
np.save('labels_all.npy',np.array(labels))
np.save('subs_all.npy',np.array(subjects))
np.save('trial_all.npy',np.array(sessions))

### Save in .mat format (MATLAB)

In [13]:
from scipy.io import savemat

# save the whole dataset in .mat
dataset_dict = {'emg':global_emg_segments, 'imu':global_imu_segments, 'labels':labels, 'subjects':subjects, 'trials':sessions}
savemat('dataset.mat', dataset_dict)